In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

In [6]:
OUTPUT_fig = 'output/fig'
OUTPUT_csv = 'output/csv'
os.makedirs(OUTPUT_csv, exist_ok=True)
os.makedirs(OUTPUT_fig, exist_ok=True)

#train data set
logo_df = pd.read_csv('data/logo.csv')
x_train = logo_df.to_numpy()
t_train = np.zeros((x_train.shape[0], 1))

class dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.astype('float32')
        self.t_train = t_train
    
    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(t_train[idx], dtype=torch.long)

trainval_data = dataset(x_train, t_train)


In [7]:
batch_size = 32
val_size = 1000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size = batch_size,
    shuffle = True
)
dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size = batch_size,
    shuffle = True
)

In [4]:
deepSDF = nn.Sequential(
    nn.Linear(3, 2),
    nn.BatchNorm1d(2),
    nn.ReLU(),
    nn.Linear(2, 2),
    nn.BatchNorm1d(2),
    nn.ReLU(),
    nn.Linear(2, 1),
    nn.BatchNorm1d(1),
    nn.Tanh()
)

def init_weights_xav(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.0)

def init_weights_he(m):  # He
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)

deepSDF.apply(init_weights_he)

def loss_function(y, t, delta=0.1):
    y_cl = torch.clamp(y, min=-delta, max=delta)
    t_cl = torch.clamp(t, min=-delta, max=delta)

    return torch.abs(y_cl - t_cl).sum()


lr = 0.1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
deepSDF.to(device)
optimizer = optim.Adam(deepSDF.parameters(), lr=lr)


In [5]:
n_epochs = 20

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    deepSDF.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]
        deepSDF.zero_grad()
        x = x.to(device)
        t = t.to(device)
        y = deepSDF.forward(x)
        loss = loss_function(y, t)

        loss.backward()
        optimizer.step()

        acc_train += (torch.abs(y) < torch.tensor(0.05)).float().sum().item()
        losses_train.append(loss.tolist())

    deepSDF.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]
        deepSDF.zero_grad()
        x = x.to(device)
        t = t.to(device)
        y = deepSDF.forward(x)
        loss = loss_function(y, t)

        acc_val += (torch.abs(y) < torch.tensor(0.05)).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))  
        




EPOCH: 0, Train [Loss: 2.732, Accuracy: 528.000], Valid [Loss: 3.100, Accuracy: 0.007]
EPOCH: 1, Train [Loss: 3.159, Accuracy: 56.000], Valid [Loss: 3.078, Accuracy: 0.016]
EPOCH: 2, Train [Loss: 3.075, Accuracy: 144.000], Valid [Loss: 3.057, Accuracy: 0.021]
EPOCH: 3, Train [Loss: 3.151, Accuracy: 62.000], Valid [Loss: 3.108, Accuracy: 0.006]
EPOCH: 4, Train [Loss: 3.132, Accuracy: 89.000], Valid [Loss: 3.068, Accuracy: 0.019]
EPOCH: 5, Train [Loss: 3.108, Accuracy: 118.000], Valid [Loss: 2.983, Accuracy: 0.046]
EPOCH: 6, Train [Loss: 2.658, Accuracy: 718.000], Valid [Loss: 2.806, Accuracy: 0.104]
EPOCH: 7, Train [Loss: 3.128, Accuracy: 98.000], Valid [Loss: 3.125, Accuracy: 0.000]
EPOCH: 8, Train [Loss: 3.200, Accuracy: 0.000], Valid [Loss: 3.125, Accuracy: 0.000]
EPOCH: 9, Train [Loss: 3.200, Accuracy: 0.000], Valid [Loss: 3.125, Accuracy: 0.000]
EPOCH: 10, Train [Loss: 3.200, Accuracy: 0.000], Valid [Loss: 3.125, Accuracy: 0.000]
EPOCH: 11, Train [Loss: 3.200, Accuracy: 0.000], Val

In [9]:
deepSDF(torch.tensor(x_train[1],dtype=torch.float))
torch.tensor(x_train[1],dtype=torch.float)

ValueError: expected 2D or 3D input (got 1D input)